Source: [Tutorial](https://worldbank.github.io/OpenNightLights/tutorials/mod6_3_intro_to_sentinel2.html)

[sentinel-2](https://sentinel.esa.int/web/sentinel/missions/sentinel-2)

[(GEE) collection’s landing pages](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2) -- more details


#### Intro

##### Sentinel-2 satellites
* hi-resolution imagery provided by the MultiSpectral Instrument (MSI), useful for land use monitoring
* * part of European Space Agency’s (ESA) Copernicus system

* several image bands across the *optical* electromagnetic spectrum, useful  for classifying land use, particular that of built-up areas.

* * For visualization: just RGB channels

In [1]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal
import geemap, ee
import os

In [2]:
ee.Initialize()

# get our admin boundary
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME','Ireland')).geometry()

# Sentinel-2 image filtered on 2019 and on Ireland
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate("2019-01-01","2019-12-31").filterBounds(aoi).median().divide(10000)

# channels
rgb = ['B4','B3','B2']

# set some thresholds
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}

In [3]:
# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 7)
map1.addLayer(se2.clip(aoi), rgbViz, "S2")

map1.addLayerControl()

### attempt 1: save map (GUI)

![Ireland - cloud cover](./IRL_clouds.png)

### [attempt 2](https://geemap.org/notebooks/11_export_image/#export-an-image-to-google-drive)

In [4]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'IRL_clouds.tif')

In [14]:
feature = map1.draw_last_feature
if feature is None:
    geom = ee.Geometry.Polygon([[[-115.413031, 35.889467],
    [-115.413031, 36.543157],
    [-114.034328, 36.543157],
    [-114.034328, 35.889467],
    [-115.413031, 35.889467]]])
    feature = ee.Feature(geom, {})

roi = feature.geometry()

# Exporting all bands as one single image
se2 = se2.clip(roi).unmask()
geemap.ee_export_image(se2, filename=filename, scale=90, region=roi, file_per_band=False)

Issue: It appears we’ve also captured some clouds. 

Solution: We will make a cloud mask to clear the image up using Sentinel-2’s QA band. We’re modeling this (in Python) from the [example used in GEE](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2#bands)
* Name: `QA60`
* * Description: Cloud Mask

In [5]:
def se2mask(image):
    quality_band = image.select('QA60')
    
    # using the bit mask for clouds, incl cirrus clouds
    cloudmask = 1 << 10
    cirrusmask = 1 << 11
    
    # only clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
    
    # divide by 10000 to make interpreting reflectance values easier
    return image.updateMask(mask).divide(10000)
    

In [6]:
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    "2019-01-01","2019-12-31").filterBounds(aoi).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",20)).map(se2mask).median()

# initialize our map
map2 = geemap.Map()
map2.centerObject(aoi, 7)
map2.addLayer(se2.clip(aoi), rgbViz, "S2")

map2.addLayerControl()
map2

Map(center=[53.16757734522266, -8.160256013964048], controls=(WidgetControl(options=['position', 'transparent_…